# Overview of GPS guided Robot code

In [ ]:
def convert_to_degrees(raw_value):
    decimal_value = raw_value/100.00
    degrees = int(decimal_value)
    mm_mmmm = (decimal_value - int(decimal_value))/0.6
    position = degrees + mm_mmmm 
#     position = "%.4f" %(position)
    return position

Get current Headding/Facing direction from Magnetometer (0 to 360 deg)

In [ ]:
# Magneto Meter

def currentHeading():
    
    imu.readSensor()
    imu.computeOrientation()
    
        
    magX = imu.MagVals[0] 
    magY = imu.MagVals[1]
    M_PI = math.pi
    offSet = 30 # Correction Angle
    
    if (magY > 0):
        heading = 90 - math.atan(magX/magY)*(180/M_PI)
        
    elif (magY < 0):
        heading = 270 - math.atan(magX/magY)*(180/M_PI)
        
    elif (magY == 0 and magX < 0):
        heading = 180
        
    elif (magY == 0 and magX > 0): 
        heading = 0
        
    heading = -heading + offSet
    if heading > 360:
        heading = heading - 360;
    elif heading < 0:
        heading = heading + 360;
        
    return heading

Get Waypoint when target heading, distance and current location is given

In [ ]:
# Navigation
def getWayPoint(currHead, targHead, currLat, currLon, dist): # Get Waypoint when target heading, distance and current location is given
    R = 6378.137 * 1000  # Radius of earth in M 
    
    Ad = dist/R * 180/math.pi # Angular distance
    
    targHead = currHead + targHead # previous targHead is w.r.t North (Bearing Angle)
    
    targLat = np.rad2deg(math.asin(math.sin(np.deg2rad(currLat)) * math.cos(np.deg2rad(Ad)) + math.cos(np.deg2rad(currLat)) * math.sin(np.deg2rad(Ad)) * math.cos(np.deg2rad(targHead)))) 
    targLong = currLon + np.rad2deg(math.atan2(math.sin(np.deg2rad(targHead)) * math.sin(np.deg2rad(Ad)) * math.cos(np.deg2rad(currLat)), math.cos(np.deg2rad(Ad)) - math.sin(np.deg2rad(currLat)) * math.sin(np.deg2rad(targLat))))
    
    return [targLat,targLong]

Steering angle towards target locaation wrt target location

In [ ]:
def getSteering(currHead, currLat, currLon, targLat, targLon):
    # currHead must be from -360 to 360;
    x = math.cos(np.deg2rad(targLat)) * math.sin(np.deg2rad(currLon-targLon));
    y = math.cos(np.deg2rad(currLat)) * math.sin(np.deg2rad(targLat)) - math.sin(np.deg2rad(currLat))* math.cos(np.deg2rad(targLat)) * math.cos(np.deg2rad(currLon-targLon));
    mag = currHead;
    steering = -np.rad2deg(math.atan2(x,y)) - mag;
    if steering >= 180:
        steering = steering - 360;
    elif steering <= -180:
        steering = steering + 360;
    return steering

Distance between current location and final target

In [ ]:
def getDistance(curLat, curLon, lat2, lon2)
    R = 6378.137; # Radius of earth in KM
    dLat = lat2 * math.pi / 180 - curLat * math.pi / 180
    dLon = lon2 * math.pi / 180 - curLon * math.pi / 180
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(curLat * math.pi / 180) * math.cos(lat2 * math.pi / 180) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c;
    return d * 1000; # meters

Function for Differential Driving for the robot to move and steer towards the target dynamically

In [ ]:
def mapf(x, in_min, in_max, out_min, out_max):
    return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

def differentialDrive(steeringAngle, driveSpeed):
    # steeringAngle must be from -180 to 180.

    leftDriveSpeed = driveSpeed;
    rightDriveSpeed = driveSpeed;
    if (steeringAngle <= 0):
        L = leftDriveSpeed - mapf(abs(steeringAngle), 0,180, 0,driveSpeed) # slow left down
        R = rightDriveSpeed
        
    if (steeringAngle < -90) and (steeringAngle >= -180):
        L = leftDriveSpeed - 2*mapf(abs(steeringAngle), 0,180, 0,driveSpeed) # slow left down
        R = rightDriveSpeed
      
    
    if (steeringAngle > 0) and steeringAngle <= 90:
        L = leftDriveSpeed
        R = rightDriveSpeed - mapf(abs(steeringAngle), 0,180, 0,driveSpeed) # slow right down
    
    if (steeringAngle > 90) and steeringAngle > 90:
        L = leftDriveSpeed
        R = rightDriveSpeed - 2*mapf(abs(steeringAngle), 0,180, 0,driveSpeed) # slow right down
        

    L_R_velocity = [L, R,mapf(abs(steeringAngle), 0,180, 0,driveSpeed)]
    return L_R_velocity